# How to use LangChain and Azure OpenAI with Python


Langchain is an open source framework for developing applications using large language models (LLM). <br>

This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [1]:
#%pip install --upgrade openai
%pip install langchain

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



## API Configuation and Deployed Model Setup

After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment. 


In [8]:
import openai
import json
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain import LLMChain


# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base=config_details['OPENAI_API_BASE']
    
# API version e.g. "2023-07-01-preview"
openai_api_version=config_details['OPENAI_API_VERSION']

# The name of your Azure OpenAI deployment chat model. e.g. "gpt-35-turbo-0613"
deployment_name=config_details['DEPLOYMENT_NAME']

# The API key for your Azure OpenAI resource.
openai_api_key = config_details["openai_api_key"]

# This is set to `azure`
openai_api_type="azure"

## Deployed Model Setup

In [9]:
# Create an instance of chat llm
llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version=openai_api_version,
    deployment_name=deployment_name,
    openai_api_key=openai_api_key,
    openai_api_type=openai_api_type,
)

llm([HumanMessage(content="Write me a poem")])

AIMessage(content="In the depths of the night,\nAs the moon shines so bright,\nMy mind wanders and roams,\nAs I sit here all alone.\n\nThe stars twinkle above,\nThe world sleeps like a dove,\nBut my thoughts, they never cease,\nAs I try to find some peace.\n\nThe darkness envelops me,\nIt's where I am truly free,\nTo explore the unknown,\nAnd make it my very own.\n\nI hear the whispers of the wind,\nAs it caresses my skin,\nAnd I know I'm not alone,\nIn this world that I call home.\n\nFor nature is my companion,\nIn this midnight exploration,\nAnd together we can create,\nA world that's truly great.\n\nThe night can sometimes be scary,\nBut it's also truly merry,\nAs I find solace in my thoughts,\nAnd the secrets that it holds.\n\nSo I sit here and ponder,\nAs the world outside continues to wander,\nFor in the darkness I find,\nThe light that guides my mind.", additional_kwargs={})

## PromptTemplates

Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [10]:
from langchain import PromptTemplate

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


# First Example
template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
input_variables=["product_type", "customer_request"],
template=template,
)

print("Example #1:")
print(llm([HumanMessage(content=prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    ))]
))
print("\n")

# Second Example
system_message = "You are an AI assistant travel assistant that provides vacation recommendations."

system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(f"Where should I go on vaction in Decemember for warm weather and beaches?")
print("Example #2:")
print(result)

Example #1:
content="For acne-prone skin, I would recommend a face wash that contains salicylic acid or benzoyl peroxide as they are effective in unclogging pores and killing acne-causing bacteria. Some of the recommended face washes for acne-prone skin are Neutrogena Oil-Free Acne Wash, La Roche-Posay Effaclar Medicated Gel Cleanser, and CeraVe Foaming Facial Cleanser. However, it is important to note that everyone's skin is different, and it is best to consult with a dermatologist to determine the best regimen for your specific skin concerns." additional_kwargs={}


Example #2:
There are plenty of beautiful destinations that offer warm weather and sandy beaches in December. Here are some recommendations:

1. The Maldives- The Maldives is a postcard-perfect destination with turquoise lagoons, palm trees, and white sand beaches. It's a great place to relax and unwind as you soak up the sun.

2. Phuket, Thailand- Phuket is famous for its beautiful beaches, parties and nightlife, and stu

## Chains
There are many applications of chains that allow you to combine numerous LLM calls and actions.  <br>

### Simple Sequential Chains  <br>
Allow you to feed the output of one LLM Chain as input for another.

In [11]:
from langchain.chains import SimpleSequentialChain

In [12]:
description_template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
description_prompt_template = PromptTemplate(input_variables=["child_description"], template=description_template)

description_chain = LLMChain(llm=llm, prompt=description_prompt_template)

In [13]:
diy_description_template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
diy_prompt_template = PromptTemplate(input_variables=["diy_project"], template=diy_description_template)

diy_chain = LLMChain(llm=llm, prompt=diy_prompt_template)

In [14]:
overall_chain = SimpleSequentialChain(chains=[description_chain, diy_chain], verbose=True)

In [15]:
review = overall_chain.run("5-year-old girl")



> Entering new SimpleSequentialChain chain...
A fun DIY project for a 5-year-old girl can be making a personalized handprint or footprint flower garden with paint. All you need is a canvas, paint, and some flowers print-outs. You can help her dip her hand or foot into the paint and press it onto the canvas to create flower petals. Then, she can use brushes to add some details like stems and leaves. Once everything is dried, you can glue on the flower print-outs to complete her beautiful garden masterpiece. It's a great memory to keep and can also be hung up as a wall decoration in her room.
Materials:
- Canvas
- Paint (assorted colors)
- Paintbrushes
- Flower print-outs
- Glue

Instructions:
1. Lay down a protective surface on your workspace, such as newspaper or a plastic sheet.
2. Help your child choose a color for the petals and pour a small amount onto a palette or plate.
3. Have your child dip their hand or foot into the paint and press it onto the canvas, creating petals. Repea